In [1]:
import mo_descriptor as md
import nn_frame as nn
import numpy as np
import subprocess
subprocess.run('export TF_INTRA_OP_PARALLELISM_THREADS=12', shell=True)

2022-07-12 13:39:49.590628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-12 13:39:49.590673: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CompletedProcess(args='export TF_INTRA_OP_PARALLELISM_THREADS=12', returncode=0)

In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i)
#         homo_pairs[idx][2] = np.add(homo_pair[2],j)
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('data/cdft-V1V2.dat')
c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2

In [4]:
train_homo_pairs = homo_pairs[0:1200]
train_lumo_pairs = lumo_pairs[0:1200]

train_c_homo = c_homo[0:1200].reshape((len(c_homo[0:1200]),1))
train_c_lumo = c_lumo[0:1200]

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:].reshape((len(c_homo[1200:]),1))
test_c_lumo = c_lumo[1200:]

In [12]:
setting = {'activation':'tanh', 'nn_shape':(240,240,240), 'batch_size':80, 'training_steps':200000,\
'learning_rate': 0.005, 'decay_rate':0.90, 'decay_per_steps':2000, 'save_step':10000}
NN = nn.NN(setting_dict=setting)
NN.train(train_homo_pairs,train_c_homo)
NN.test(test_homo_pairs, test_c_homo)

training step:     0, loss: 0.023386
training step: 10000, loss: 0.026573
training step: 20000, loss: 0.020294
training step: 30000, loss: 0.013476
training step: 40000, loss: 0.006518
training step: 50000, loss: 0.002669
training step: 60000, loss: 0.001043
training step: 70000, loss: 0.000418
training step: 80000, loss: 0.000192
training step: 90000, loss: 0.000111
training step: 100000, loss: 0.000083
training step: 110000, loss: 0.000073
training step: 120000, loss: 0.000069
training step: 130000, loss: 0.000068
training step: 140000, loss: 0.000068
training step: 150000, loss: 0.000068
training step: 160000, loss: 0.000068
training step: 170000, loss: 0.000068
training step: 180000, loss: 0.000068
training step: 190000, loss: 0.000068
INFO:tensorflow:Assets written to: ./save/model/assets
88 (80, 1) (80, 1)
88 (80, 1) (80, 1)
88 (80, 1) (80, 1)
88 (80, 1) (80, 1)
88 (80, 1) (80, 1)
88 (1, 1) (1, 1)
88 (80, 1) (80, 1)


<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [13]:
print((NN.model(test_homo_pairs)-test_c_homo)/test_c_homo)

tf.Tensor(
[[-0.8097773 ]
 [-0.77503395]
 [-0.7249728 ]
 [-0.64755297]
 [-0.51388234]
 [-0.23187564]
 [ 0.7381034 ]
 [ 7.4487824 ]
 [ 0.29039884]
 [-0.2801798 ]
 [-0.4887522 ]
 [-0.5951477 ]
 [-0.6583762 ]
 [-0.69928724]
 [-0.7270288 ]
 [-0.7461939 ]
 [-0.75934047]
 [-0.7680086 ]
 [-0.7731216 ]
 [-0.77538824]
 [-0.7752014 ]
 [-0.7727648 ]
 [-0.7682141 ]
 [-0.7616126 ]
 [-0.7529383 ]
 [-0.7421125 ]
 [-0.7290233 ]
 [-0.71352774]
 [-0.6954742 ]
 [-0.67443675]
 [-0.91652447]
 [-0.9160227 ]
 [-0.9145133 ]
 [-0.91196924]
 [-0.90831035]
 [-0.90337807]
 [-0.89693475]
 [-0.8886202 ]
 [-0.87791383]
 [-0.8641249 ]
 [-0.8461938 ]
 [-0.8223359 ]
 [-0.7894536 ]
 [-0.74175775]
 [-0.6672069 ]
 [-0.5363926 ]
 [-0.25205258]
 [ 0.81947446]
 [ 3.96549   ]
 [ 0.09681346]
 [-0.3653974 ]
 [-0.54280955]
 [-0.635042  ]
 [-0.69040865]
 [-0.72642195]
 [-0.75089914]
 [-0.76780266]
 [-0.7794009 ]
 [-0.78696316]
 [-0.7913782 ]
 [-0.79322046]
 [-0.7928092 ]
 [-0.79033834]
 [-0.7859725 ]
 [-0.77976495]
 [-0.7716781 ]